In [1]:
!pip install -q transformers torch

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="NousResearch/Hermes-3-Llama-3.1-8B")



2024-11-23 08:47:05.205492: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 08:47:05.246499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-23 08:47:06.051341: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
pipe(messages)

Checking memory

In [ ]:
import torch

print(torch.cuda.memory_allocated() / (1024 ** 3), "GB allocated after loading")


In [ ]:
from pydantic import BaseModel, ValidationError
import json

class StudentProfile(BaseModel):
    major: str
    bio: str

def validate_json(response: str):
    try:
        # Attempt to parse and validate the response as a StudentProfile
        student_profile = StudentProfile.parse_raw(response)
        return student_profile
    except ValidationError as e:
        # Catch invalid JSON or schema errors and return None
        print(f"Invalid JSON or structure: {e}")
        return None
    except json.JSONDecodeError as e:
        # Catch invalid JSON formatting errors
        print(f"JSON Decode Error: {e}")
        return None

In [ ]:
def generate_response(messages):
    # Generate a response from pipeline 

    

In [ ]:
prompt_template = """
You are tasked with generating a college student profile for the "{major}" major in valid JSON format.

Requirements:
The profile must include the following fields:
"major": "{major}"
"bio": A detailed description of the student's activities, achievements, projects, leadership positions, internships, research, and goals. The bio must not contain single quotes (') or unescaped special characters.

2. Strict Formatting Rules:
    
All keys and string values 
must use double quotes (").
    
Ensure proper placement of all braces ({{}}, {{}}), commas (,), and other JSON syntax.
The output must be a complete and valid JSON object.

3. Output Restrictions:
    
Do not include names, gender, school, or any other personal identifiers.
Avoid extra text or commentary outside the JSON object.

4. Error Handling:
    
If valid JSON cannot be generated, return this exact string: "Invalid JSON".

Example of Valid JSON:
{
    "major": "{major}",
    "bio": "A detailed description of the student's activities, achievements, projects, leadership positions, internships, research, and goals."
}

Task:
Generate a JSON object containing exactly ONE student profile** for the "{major}" major. Ensure the JSON is properly formatted and valid. If you cannot generate valid JSON, return "Invalid JSON".
"""

major = "Computer Science"
formatted_prompt = prompt_template.format(major=major)